In [36]:
import os
os.environ['KERAS_BACKEND']='tensorflow'
from keras.datasets import mnist
(x_train, labels_train), (x_test, labels_test) = mnist.load_data()
x_train = x_train.astype('float32') 
x_test = x_test.astype('float32') 
x_train /= 255 
x_test /= 255
from keras.utils import to_categorical 
y_train = to_categorical(labels_train, 10) 
y_test = to_categorical(labels_test, 10)
x_train = x_train.reshape(60000, 784) 
x_test = x_test.reshape(10000, 784)
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [40]:
import keras
from keras.utils import np_utils
from keras.models import Sequential  
from keras.layers import Input, Dense, Dropout, Activation,BatchNormalization
from keras.models import Model
from keras.optimizers import SGD
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}
 
    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))
 
    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))
 
    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()

history = LossHistory()

In [42]:
net = Sequential()
net.add(Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)))
net.add(BatchNormalization())
net.add(MaxPooling2D(pool_size=(2, 2),strides=1))
net.add(Conv2D(64,(3,3), activation='relu'))
net.add(BatchNormalization())
net.add(MaxPooling2D(pool_size=(2, 2)))
net.add(Conv2D(128,(3,3), activation='relu'))
net.add(BatchNormalization())
net.add(MaxPooling2D(pool_size=(2, 2)))
net.add(Conv2D(256,(3,3), activation='relu'))
net.add(BatchNormalization())
net.add(MaxPooling2D(pool_size=(2, 2)))
net.add(Flatten())

net.add(Dense(500)) 
net.add(BatchNormalization())
net.add(Activation('relu')) 
net.add(Dropout(0.5)) 

net.add(Dense(500)) 
net.add(Activation('relu'))

net.add(Dense(10))
net.add(BatchNormalization())
net.add(Activation('softmax'))
net.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [51]:
from keras.callbacks import LearningRateScheduler
from keras import backend as K
def scheduler(epoch):
    if epoch % 5 == 0 and epoch != 0:
        lr = K.get_value(net.optimizer.lr)
        K.set_value(net.optimizer.lr,lr*0.5)
        print("learning rate changed to {}".format(lr*0.5))
    return K.get_value(net.optimizer.lr)
reduce_lr = LearningRateScheduler(scheduler)

In [49]:
Data = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

In [55]:
net.fit_generator(
    Data.flow(x_train,y_train,batch_size = 100),
    epochs = 10,
    validation_data=(x_test,y_test),
    verbose=1,
    steps_per_epoch=x_train.shape[0]//100,
    callbacks=[reduce_lr]
)

Epoch 1/10
600/600 [==============================] - 141s 235ms/step - loss: 0.0063 - accuracy: 0.9983 - val_loss: 0.0135 - val_accuracy: 0.9957
Epoch 2/10
600/600 [==============================] - 144s 240ms/step - loss: 0.0052 - accuracy: 0.9987 - val_loss: 0.0166 - val_accuracy: 0.9951
Epoch 3/10
600/600 [==============================] - 145s 241ms/step - loss: 0.0056 - accuracy: 0.9985 - val_loss: 0.0130 - val_accuracy: 0.9964
Epoch 4/10
600/600 [==============================] - 144s 240ms/step - loss: 0.0051 - accuracy: 0.9986 - val_loss: 0.0163 - val_accuracy: 0.9953
Epoch 5/10
600/600 [==============================] - 144s 240ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.0157 - val_accuracy: 0.9959
Epoch 6/10
learning rate changed to 0.0002500000118743628
600/600 [==============================] - 141s 236ms/step - loss: 0.0047 - accuracy: 0.9986 - val_loss: 0.0158 - val_accuracy: 0.9958
Epoch 7/10
600/600 [==============================] - 142s 236ms/step - loss:

In [56]:
# save model
net.save('test2.h5')